In [8]:
import pandas as pd
import numpy as np

from pathlib import Path
from settings import config

DATA_DIR = Path(config("DATA_DIR"))
MANUAL_DATA_DIR = Path(config("MANUAL_DATA_DIR"))

from pull_match_list import load_match_list

df = load_match_list()

In [17]:
from load_player_id import load_player_id_df

player_df = load_player_id_df()

In [54]:
df["duration_minutes"] = df["Duration"].str.split(":").apply(lambda x: int(x[0]) + int(x[1]) / 60)

AttributeError: 'list' object has no attribute 'str'

In [124]:
red_team = ["theshy","jiejie","rookie","gala","meiko"]
player_df["player_lower"] = player_df["player"].str.lower().str.strip()  
player_df[player_df["player_lower"].isin(red_team)]["player"].unique().tolist()


['GALA', 'Jiejie', 'RooKie', 'TheShy', 'meiko']

In [69]:
df = calc_kda(df,"Score","")

In [99]:
blue_team = ["Bin","Wei","Knight","Elk","ON"]
red_team = ["TheShy","Jiejie","RooKie","GALA","meiko"] # for reference, this is the red team in the first match of 2023 Worlds

def shared_games_list(blue_team, red_team, match_df):
    result = []
    for blue, red in zip(blue_team, red_team):
        blue_df = match_df[match_df["player"] == blue]
        red_df  = match_df[match_df["player"] == red]
        merged_df = pd.merge(blue_df, red_df, on=["Date", "Game","Duration"],suffixes=('_blue', '_red'))
        result.append(merged_df)
    return result


In [100]:
duizhan_df_list = shared_games_list(blue_team, red_team, df)

In [59]:
def calc_kda(df,score_column_name,target_column_suffix):
    kda_split = df[score_column_name].str.strip().str.split("/", expand=True)
    df[f"K{target_column_suffix}"] = kda_split[0].astype(int)
    df[f"D{target_column_suffix}"] = kda_split[1].astype(int)
    df[f"A{target_column_suffix}"] = kda_split[2].astype(int)
    return df

In [101]:
df.head(5)

,Champion,Result,Score,Build,Duration,Date,Game,Tournament,player,gg_id,season,duration_minutes,K,D,A
0,Rumble,Defeat,2/2/1,,29:39,2024-08-31,SHG vs FAK,PCS Summer Playoffs 2024,1Jiang,5421,S14,29.650000,2,2,1
1,Rumble,Defeat,3/4/7,,38:23,2024-08-31,SHG vs FAK,PCS Summer Playoffs 2024,1Jiang,5421,S14,38.383333,3,4,7
2,Riven,Defeat,2/4/2,,30:01,2024-08-31,SHG vs FAK,PCS Summer Playoffs 2024,1Jiang,5421,S14,30.016667,2,4,2
3,Aatrox,Victory,1/0/5,,28:01,2024-08-31,SHG vs FAK,PCS Summer Playoffs 2024,1Jiang,5421,S14,28.016667,1,0,5
4,Riven,Victory,2/6/6,,36:57,2024-08-24,FAK vs CFO,PCS Summer Playoffs 2024,1Jiang,5421,S14,36.950000,2,6,6


In [102]:
duizhan_df_list[0].head(5)

,Champion_blue,Result_blue,Score_blue,Build_blue,Duration,Date,Game,Tournament_blue,player_blue,gg_id_blue,...,Score_red,Build_red,Tournament_red,player_red,gg_id_red,season_red,duration_minutes_red,K_red,D_red,A_red
0,Camille,Defeat,5/4/4,,38:45,2020-06-14,Suning vs IG,LPL Summer 2020,Bin,2606,...,3/6/6,,LPL Summer 2020,TheShy,1075,S10,38.750000,3,6,6
1,Aatrox,Defeat,0/2/0,,24:10,2020-06-14,Suning vs IG,LPL Summer 2020,Bin,2606,...,3/1/2,,LPL Summer 2020,TheShy,1075,S10,24.166667,3,1,2
2,Renekton,Defeat,4/3/3,,24:44,2020-03-28,IG vs Suning,LPL Spring 2020,Bin,2606,...,5/6/5,,LPL Spring 2020,TheShy,1075,S10,24.733333,5,6,5
3,Jayce,Victory,1/4/2,,27:24,2021-02-28,Suning vs IG,LPL Spring 2021,Bin,2606,...,2/3/3,,LPL Spring 2021,TheShy,1075,S11,27.400000,2,3,3
4,Renekton,Victory,4/1/11,,28:50,2021-02-28,Suning vs IG,LPL Spring 2021,Bin,2606,...,1/6/0,,LPL Spring 2021,TheShy,1075,S11,28.833333,1,6,0


In [103]:
def generate_compete_summary_df(lst, top_n_heroes=3):
    summary_list = []

    for df in lst:
        result = df.groupby(["player_blue", "player_red"]).apply(
            lambda group: pd.Series({
                "蓝方胜场": int((group["Result_blue"] == "Victory").sum()),
                "红方胜场": int((group["Result_red"] == "Victory").sum()),

                "蓝方平均K": group["K_blue"].mean(),
                "蓝方平均D": group["D_blue"].mean(),
                "蓝方平均A": group["A_blue"].mean(),

                "红方平均K": group["K_red"].mean(),
                "红方平均D": group["D_red"].mean(),
                "红方平均A": group["A_red"].mean(),

                "蓝方常用英雄": ", ".join(group["Champion_blue"].value_counts().head(top_n_heroes).index),
                "红方常用英雄": ", ".join(group["Champion_red"].value_counts().head(top_n_heroes).index),

                "平均小局时长": group["duration_minutes_blue"].mean(),
                "总小局数": int(len(group)),
                "总大局数": int(group[["Game", "Date"]].drop_duplicates().shape[0]),
            })
        ).reset_index()

        # 胜率列
        result["蓝方小局胜率"] = result["蓝方胜场"] / result["总小局数"]
        result["红方小局胜率"] = result["红方胜场"] / result["总小局数"]

        # KDA列
        result["蓝方小局KDA"] = result.apply(
            lambda row: f"{row['蓝方平均K']:.2f}/{row['蓝方平均D']:.2f}/{row['蓝方平均A']:.2f}", axis=1
        )
        result["红方小局KDA"] = result.apply(
            lambda row: f"{row['红方平均K']:.2f}/{row['红方平均D']:.2f}/{row['红方平均A']:.2f}", axis=1
        )

        # 胜场(胜率)
        result["蓝方胜场(胜率)"] = result.apply(
            lambda row: f"{row['蓝方胜场']} ({row['蓝方小局胜率']:.2%})", axis=1
        )
        result["红方胜场(胜率)"] = result.apply(
            lambda row: f"{row['红方胜场']} ({row['红方小局胜率']:.2%})", axis=1
        )

        # 重命名选手列
        result = result.rename(columns={
            "player_blue": "蓝方选手",
            "player_red": "红方选手"
        })

        # 删除中间列
        result = result.drop(columns=[
            "蓝方平均K", "蓝方平均D", "蓝方平均A",
            "红方平均K", "红方平均D", "红方平均A",
            "蓝方小局胜率", "红方小局胜率"
        ])

        # 重新排布列顺序（对称结构）
        column_order = [
            "蓝方选手", "蓝方胜场", "蓝方胜场(胜率)", "蓝方小局KDA", "蓝方常用英雄",
            "总小局数", "总大局数", "平均小局时长",
            "红方常用英雄", "红方小局KDA", "红方胜场(胜率)", "红方胜场", "红方选手"
        ]
        result = result[column_order]

        summary_list.append(result)

    final_df = pd.concat(summary_list, ignore_index=True)
    return final_df


In [104]:
generate_compete_summary_df(duizhan_df_list)

C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_12932\875560668.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(["player_blue", "player_red"]).apply(
C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_12932\875560668.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = df.groupby(["player_blue", "player_red"]).apply(
C:\Users\Yangge Xu\AppData\Local\Temp\ipykernel_12932\87556066

,蓝方选手,蓝方胜场,蓝方胜场(胜率),蓝方小局KDA,蓝方常用英雄,总小局数,总大局数,平均小局时长,红方常用英雄,红方小局KDA,红方胜场(胜率),红方胜场,红方选手
0,Bin,14,14 (58.33%),2.83/3.04/4.75,"Kennen, Gnar, Aatrox",24,9,30.954167,"KSante, Renekton, Jayce",2.71/3.96/3.67,10 (41.67%),10,TheShy
1,Wei,17,17 (48.57%),2.49/2.74/6.03,"Xin Zhao, Wukong, Olaf",35,12,31.591905,"Hecarim, Vi, Maokai",2.03/2.20/6.66,18 (51.43%),18,Jiejie
2,Knight,26,26 (60.47%),4.28/1.91/5.56,"Syndra, Tristana, Ryze",43,15,31.246512,"LeBlanc, Orianna, Syndra",2.95/2.60/5.28,17 (39.53%),17,RooKie
3,Elk,14,14 (41.18%),3.35/2.59/4.82,"Aphelios, Ezreal, Jinx",34,14,30.863725,"Zeri, Aphelios, Varus",4.88/1.65/5.06,20 (58.82%),20,GALA
4,ON,25,25 (58.14%),1.16/3.49/9.44,"Rakan, Rell, Leona",43,13,32.173256,"Nautilus, Alistar, Rakan",0.86/3.51/8.07,18 (41.86%),18,meiko


In [115]:
def generate_hero_summary_df(lst):
    hero_summary_list = []

    for df in lst:
        df["player_pair"] = df.apply(lambda row: tuple(sorted([row["player_blue"], row["player_red"]])), axis=1)

        for pair, match_df in df.groupby("player_pair"):
            blue, red = pair

            blue_heroes = match_df["Champion_blue"]
            red_heroes = match_df["Champion_red"]

            blue_counts = blue_heroes.value_counts()
            red_counts = red_heroes.value_counts()

            blue_win_rate = match_df.groupby("Champion_blue")["Result_blue"].apply(lambda x: (x == "Victory").mean())
            red_win_rate = match_df.groupby("Champion_red")["Result_red"].apply(lambda x: (x == "Victory").mean())

            blue_most = blue_counts.idxmax()
            red_most = red_counts.idxmax()

            blue_most_str = f"{blue_most} ({blue_counts[blue_most]}/{blue_win_rate[blue_most]*100:.1f}%)"
            red_most_str = f"{red_most} ({red_counts[red_most]}/{red_win_rate[red_most]*100:.1f}%)"

            blue_best = blue_win_rate.idxmax()
            blue_worst = blue_win_rate.idxmin()

            red_best = red_win_rate.idxmax()
            red_worst = red_win_rate.idxmin()

            blue_best_str = f"{blue_best} ({blue_counts[blue_best]}/{blue_win_rate[blue_best]*100:.1f}%)"
            blue_worst_str = f"{blue_worst} ({blue_counts[blue_worst]}/{blue_win_rate[blue_worst]*100:.1f}%)"

            red_best_str = f"{red_best} ({red_counts[red_best]}/{red_win_rate[red_best]*100:.1f}%)"
            red_worst_str = f"{red_worst} ({red_counts[red_worst]}/{red_win_rate[red_worst]*100:.1f}%)"

            all_heroes = pd.concat([blue_heroes, red_heroes])
            total_unique = all_heroes.nunique()
            most_common = all_heroes.value_counts().idxmax()
            most_common_str = f"{most_common} ({all_heroes.value_counts()[most_common]})"

            hero_summary_list.append({
                "蓝方选手": blue,
                "蓝方最常用英雄": blue_most_str,
                "蓝方最高胜率英雄": blue_best_str,
                "蓝方最低胜率英雄": blue_worst_str,
                "蓝方使用英雄数": blue_heroes.nunique(),
                "对战英雄总数": total_unique,
                "对战中最常用英雄": most_common_str,
                "红方使用英雄数": red_heroes.nunique(),
                "红方最常用英雄": red_most_str,
                "红方最高胜率英雄": red_best_str,
                "红方最低胜率英雄": red_worst_str,
                "红方选手": red
            })

    df_summary = pd.DataFrame(hero_summary_list)

    # 重新排列表头，实现左右对称 + 中间公共字段
    column_order = [
        "蓝方选手",
        "蓝方最常用英雄", "蓝方最高胜率英雄", "蓝方最低胜率英雄", "蓝方使用英雄数",
        "对战英雄总数", "对战中最常用英雄",
        "红方使用英雄数", "红方最低胜率英雄","红方最高胜率英雄", "红方最常用英雄", 
        "红方选手"
    ]

    return df_summary[column_order]


In [116]:
generate_hero_summary_df(duizhan_df_list)

,蓝方选手,蓝方最常用英雄,蓝方最高胜率英雄,蓝方最低胜率英雄,蓝方使用英雄数,对战英雄总数,对战中最常用英雄,红方使用英雄数,红方最低胜率英雄,红方最高胜率英雄,红方最常用英雄,红方选手
0,Bin,Kennen (4/50.0%),Gwen (2/100.0%),Aatrox (3/0.0%),10,18,Kennen (6),15,Aatrox (1/0.0%),Gwen (1/100.0%),KSante (4/25.0%),TheShy
1,Jiejie,Xin Zhao (6/66.7%),Jarvan IV (1/100.0%),Graves (1/0.0%),16,18,Vi (8),14,Lillia (1/0.0%),Sejuani (1/100.0%),Hecarim (6/33.3%),Wei
2,Knight,Syndra (9/66.7%),Ahri (4/100.0%),LeBlanc (2/0.0%),15,26,Syndra (14),20,Aurelion Sol (2/0.0%),Galio (1/100.0%),LeBlanc (7/14.3%),RooKie
3,Elk,Aphelios (6/66.7%),Lucian (1/100.0%),Jhin (2/0.0%),11,16,Aphelios (12),13,Caitlyn (1/0.0%),Ashe (1/100.0%),Zeri (6/66.7%),GALA
4,ON,Rakan (9/77.8%),Ashe (1/100.0%),Blitzcrank (1/0.0%),15,24,Nautilus (13),18,Rakan (4/0.0%),Ashe (1/100.0%),Nautilus (8/50.0%),meiko


In [ ]:
值此第二赛段开幕之际，lz想要实践一下python的一些自动化~所以制作了以下的表格，希望大家看得开心！
暂时每场比赛的交战的两支队伍将会根据**对位选手的比赛历史记录**生成两张表格。换言之，只统计了他们的对战的数据。
有两张表格，一张是对位选手对位历史的一些简单数据，还有一个是英雄统计。

一些说明：
1. 数据来源于gol.gg，不一定准确，前面就有姐妹指出过小天缺少了一大局游戏的统计(其实是一小局，但是少了一小局就不满足BO5的筛选要求了，所以少了一大局)
2. 电脑内存和效率有限，所以只选取了在S14-15期间在LCK和LPL打过LOL的选手在S10-S15第一赛段之间的数据。
3. 现在暂时包含了所有类型比赛的数据，包括但不限于德杯，LPL，LCK，世界赛。单挑选边的数据没有计入其中。
4. 这里的蓝红方是根据接下来的比赛的赛程表决定的，左边默认蓝方。
5. 在写码的过程中用了一些AI（蓝色方红色方一个个打起来太累了，python也无法识别中文的标点符号：（），但我本人是真的会写码，也没有让AI提供任何想法和创意！AI也是根据我的码和想法来帮助提高效率的。即使用了AI辅助，也真的花了一定的时间和精力在这件事上的。
6. 数据处理还是主要用的pandas，表格可视化网上找到了一个library叫great_tables。



In [126]:
from daily_report_functions import *

from pull_match_list import load_match_list
from load_player_id import load_player_id_df
df = load_match_list()
player_df = load_player_id_df()

blue_team = ["bin","wei","knight","elk","on"]
red_team = ["theshy","jiejie","rookie","gala","meiko"]

blue, red = get_official_name(blue_team, red_team,player_df)
lst = shared_games_list(blue,red,df)
generate_compete_summary_df(lst)
generate_hero_summary_df(lst)

,蓝方选手,蓝方最常用英雄,蓝方最高胜率英雄,蓝方最低胜率英雄,蓝方使用英雄数,对战英雄总数,对战中最常用英雄,红方使用英雄数,红方最低胜率英雄,红方最高胜率英雄,红方最常用英雄,红方选手
0,Bin,Gwen (21/71.4%),Akshan (1/100.0%),Aatrox (2/0.0%),19,35,Aphelios (33),16,Caitlyn (1/0.0%),Kalista (3/100.0%),Aphelios (33/54.5%),GALA
1,Elk,Aphelios (7/57.1%),Senna (2/100.0%),Caitlyn (1/0.0%),14,27,Aphelios (7),13,Nocturne (3/0.0%),Elise (1/100.0%),Lee Sin (5/80.0%),Jiejie
2,Knight,Syndra (9/66.7%),Ahri (4/100.0%),LeBlanc (2/0.0%),15,26,Syndra (14),20,Aurelion Sol (2/0.0%),Galio (1/100.0%),LeBlanc (7/14.3%),RooKie
3,ON,Rakan (20/65.0%),Bard (2/100.0%),Braum (2/0.0%),23,49,Rakan (20),26,Aatrox (2/0.0%),Malphite (2/100.0%),Gnar (12/58.3%),TheShy
4,Wei,Xin Zhao (7/57.1%),Brand (1/100.0%),Diana (1/0.0%),17,35,Nautilus (10),18,Alistar (3/0.0%),Ashe (1/100.0%),Nautilus (10/50.0%),meiko


In [153]:
compete_sum = generate_compete_summary_df(lst)
compete_sum.head(3)

,蓝方选手,蓝方胜场,蓝方胜场(胜率),蓝方小局KDA,蓝方常用英雄,总小局数,总大局数,平均小局时长,红方常用英雄,红方小局KDA,红方胜场(胜率),红方胜场,红方选手
0,Bin,71,71 (62.83%),3.14/2.31/4.70,"Gwen, Gnar, Jax",113,50,32.170354,"Aphelios, Jinx, Xayah",4.30/1.97/5.42,70 (61.95%),70,GALA
1,Elk,13,13 (43.33%),3.60/2.47/4.83,"Aphelios, Xayah, Jinx",30,11,31.238889,"Lee Sin, Xin Zhao, Viego",2.53/2.03/7.40,17 (56.67%),17,Jiejie
2,Knight,26,26 (60.47%),4.28/1.91/5.56,"Syndra, Tristana, Ryze",43,15,31.246512,"LeBlanc, Orianna, Syndra",2.95/2.60/5.28,17 (39.53%),17,RooKie


In [156]:
df = compete_sum
df = df.rename(columns={
"蓝方选手": "选手A", "红方选手": "选手B",
"蓝方胜场(胜率)": "胜场(胜率)A", "红方胜场(胜率)": "胜场(胜率)B",
"蓝方小局KDA": "KDA_A", "红方小局KDA": "KDA_B",
"蓝方常用英雄": "常用英雄A", "红方常用英雄": "常用英雄B"
})

df = df[[
"选手A", "胜场(胜率)A", "KDA_A", "常用英雄A",
"总小局数", "总大局数", "平均小局时长",
"常用英雄B", "KDA_B", "胜场(胜率)B", "选手B"
]]
df.head(3)

,选手A,胜场(胜率)A,KDA_A,常用英雄A,总小局数,总大局数,平均小局时长,常用英雄B,KDA_B,胜场(胜率)B,选手B
0,Bin,71 (62.83%),3.14/2.31/4.70,"Gwen, Gnar, Jax",113,50,32.170354,"Aphelios, Jinx, Xayah",4.30/1.97/5.42,70 (61.95%),GALA
1,Elk,13 (43.33%),3.60/2.47/4.83,"Aphelios, Xayah, Jinx",30,11,31.238889,"Lee Sin, Xin Zhao, Viego",2.53/2.03/7.40,17 (56.67%),Jiejie
2,Knight,26 (60.47%),4.28/1.91/5.56,"Syndra, Tristana, Ryze",43,15,31.246512,"LeBlanc, Orianna, Syndra",2.95/2.60/5.28,17 (39.53%),RooKie


In [168]:
gt = display_compete_table(compete_sum,"BLG vs iG",None,"BLG","IG")
gt.save(
    "BLGvsiG.png",
    selector='table',
    scale=1.0,
    expand=5,
    web_driver='chrome',
    window_size=(6000, 6000),
    debug_port=None,
    encoding='utf-8',
    _debug_dump=None,
)

ImportError: Module selenium not found. Run the following to install.

`pip install selenium`

In [161]:
import polars as pl
import polars.selectors as cs
from great_tables import GT, loc, style


def display_compete_table(df, title=None, subtitle=None, team_a_name="队伍A", team_b_name="队伍B"):
    df = df.rename(columns={
        "蓝方选手": "选手A", "红方选手": "选手B",
        "蓝方胜场(胜率)": "胜场(胜率)A", "红方胜场(胜率)": "胜场(胜率)B",
        "蓝方小局KDA": "KDA_A", "红方小局KDA": "KDA_B",
        "蓝方常用英雄": "常用英雄A", "红方常用英雄": "常用英雄B"
    })

    df = df[[
        "选手A", "胜场(胜率)A", "KDA_A", "常用英雄A",
        "总小局数", "总大局数", "平均小局时长",
        "常用英雄B", "KDA_B", "胜场(胜率)B", "选手B"
    ]]

    gt = GT(df)

    if title or subtitle:
        gt = gt.tab_header(title=title, subtitle=subtitle)

    gt = (
        gt
        .tab_spanner(team_a_name, columns=["选手A", "胜场(胜率)A", "KDA_A", "常用英雄A"])
        .tab_spanner("对战信息", columns=["总小局数", "总大局数", "平均小局时长"])
        .tab_spanner(team_b_name, columns=["常用英雄B", "KDA_B", "胜场(胜率)B", "选手B"])
        .cols_align("center")
        .cols_label(**{
            "选手A": "选手", "选手B": "选手",
            "胜场(胜率)A": "胜场(胜率)", "胜场(胜率)B": "胜场(胜率)",
            "KDA_A": "KDA", "KDA_B": "KDA",
            "常用英雄A": "常用英雄", "常用英雄B": "常用英雄"
        })
        .tab_style(style=style.fill(color="aliceblue"), locations=loc.body(columns=["选手A", "胜场(胜率)A", "KDA_A", "常用英雄A"]))
        .tab_style(style=style.fill(color="mistyrose"), locations=loc.body(columns=["常用英雄B", "KDA_B", "胜场(胜率)B", "选手B"]))
    )

    return gt


# def display_hero_table(df, title=None, subtitle=None, team_a_name="队伍A", team_b_name="队伍B"):
#     df = df.rename(columns={
#         "蓝方选手": "选手A", "红方选手": "选手B",
#         "蓝方最常用英雄": "最常用英雄A", "红方最常用英雄": "最常用英雄B",
#         "蓝方最高胜率英雄": "胜率最高英雄A", "红方最高胜率英雄": "胜率最高英雄B",
#         "蓝方最低胜率英雄": "胜率最低英雄A", "红方最低胜率英雄": "胜率最低英雄B",
#         "蓝方使用英雄数": "使用英雄数A", "红方使用英雄数": "使用英雄数B"
#     })

#     df = df[[
#         "选手A", "最常用英雄A", "胜率最高英雄A", "胜率最低英雄A", "使用英雄数A",
#         "对战英雄总数", "对战中最常用英雄",
#         "使用英雄数B", "最常用英雄B", "胜率最高英雄B", "胜率最低英雄B", "选手B"
#     ]]

#     gt = GT(df)

#     if title or subtitle:
#         gt = gt.tab_header(title=title, subtitle=subtitle)

#     gt = (
#         gt
#         .tab_spanner(team_a_name, columns=["选手A", "最常用英雄A", "胜率最高英雄A", "胜率最低英雄A", "使用英雄数A"])
#         .tab_spanner("对战信息", columns=["对战英雄总数", "对战中最常用英雄"])
#         .tab_spanner(team_b_name, columns=["使用英雄数B", "最常用英雄B", "胜率最高英雄B", "胜率最低英雄B", "选手B"])
#         .cols_align("center")
#         .cols_label(**{
#             "选手A": "选手", "选手B": "选手",
#             "最常用英雄A": "最常用英雄", "最常用英雄B": "最常用英雄",
#             "胜率最高英雄A": "胜率最高英雄", "胜率最高英雄B": "胜率最高英雄",
#             "胜率最低英雄A": "胜率最低英雄", "胜率最低英雄B": "胜率最低英雄",
#             "使用英雄数A": "使用英雄数", "使用英雄数B": "使用英雄数"
#         })
#         .tab_style(style=style.fill(color="aliceblue"), locations=columns(["选手A", "最常用英雄A", "胜率最高英雄A", "胜率最低英雄A", "使用英雄数A"]))
#         .tab_style(style=style.fill(color="mistyrose"), locations=columns(["使用英雄数B", "最常用英雄B", "胜率最高英雄B", "胜率最低英雄B", "选手B"]))
#     )

#     return gt



